# Game Optimization Project Notebook

Authored By: Simon Lidwell

----------

This project aims to explore game winning optimization algorithms for the sake of having fun and having a deeper understanding of how to win commonly played games like Tic-Tac-Toe and Connect4.

I will start with Tic-Tac-Toe as it is a great introduction to game strategy due to the limited number of possible outcomes. Then I will move into something more complex (Connect4) and do a similar analysis.

The end of this project will be contain a comprehensive analysis of game optimization theory.

## Optimizing Tic-Tac-Toe

In [1]:
from src.utils import TicTacToe, ConnectFour

In [2]:
game1 = TicTacToe()

In [3]:
game1.get_game().model_dump()

{'id': UUID('7e9de6b2-47d3-449d-8c9a-42253bf08733'),
 'createdat': datetime.datetime(2024, 12, 5, 7, 10, 48, 883678),
 'winner': None,
 'game_type': <GameType.TIC_TAC_TOE: 'TicTacToe'>,
 'moves': []}

In [4]:
game2 = ConnectFour()

In [5]:
game2.get_game().model_dump()

{'id': UUID('f82abc64-d6df-4ce8-b290-f966366d08e7'),
 'createdat': datetime.datetime(2024, 12, 5, 7, 10, 48, 883678),
 'winner': None,
 'game_type': <GameType.CONNECT_4: 'Connect4'>,
 'moves': []}

# Game Optimization Data Analysis

So I used the backend to capture all the games and moves data into sqlite. This will allow me to observe trends in the game data.

In [18]:
from src.db.dbinit import engine

import polars as pl
from sqlalchemy import text

In [20]:
db = engine.connect()
result_df = pl.read_database(
    query=text(
        """
        SELECT *
        FROM games
            --INNER JOIN moves ON games.id = moves.game_id
        """
    ),
    connection=db,
)

db.close()

result_df

2024-12-05 03:12:42,371 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-05 03:12:42,372 INFO sqlalchemy.engine.Engine 
        SELECT *
        FROM games
            --INNER JOIN moves ON games.id = moves.game_id
        
2024-12-05 03:12:42,374 INFO sqlalchemy.engine.Engine [cached since 378.5s ago] ()
2024-12-05 03:12:42,376 INFO sqlalchemy.engine.Engine ROLLBACK


id,createdat,winner,game_type
str,str,null,str
"""89079512-3c67-44bb-9152-35b4d7…","""2024-12-05 07:10:48.883678""",null,"""TicTacToe"""
"""7e9de6b2-47d3-449d-8c9a-42253b…","""2024-12-05 07:10:48.883678""",null,"""TicTacToe"""
"""f82abc64-d6df-4ce8-b290-f96636…","""2024-12-05 07:10:48.883678""",null,"""Connect4"""
